
# DBSQL + AI Workshop: "Mining Insights from the gold layer"
This hands-on workshop is designed to empower data analysts, engineers, and business users to extract maximum value from their refined, business-ready datasets using Databricks' unified analytics platform. The "Gold Layer" refers to the curated, high-quality data tier in the medallion architecture that's optimized for analytics and reporting.

<div style="display: flex; align-items: center;">
  <img src="Images/Data + AI.png" alt="generated-image.png" style="margin-left: 10px;" width="250"/>
  <ul>
    <li>Unity Catalog managed tables: advantages and advanced uses</li>
    <li>Leverage AI SQL functions to enrich data and extract insights</li>
    <li>Correctly develop AI/BI Dashboards, use AI to facilitate your visualizations</li>
    <li>Set up a genie space to empower your business users with last-mile analytics</li>
    <li>Finally, combine Dashboards and Genie to get the full AI/BI experience</li>
  </ul>
</div>

In [0]:
%sql
USE CATALOG users; 
Use SCHEMA fernando_vasquez;

In [0]:
%sql
--SELECT * FROM fct_coffee_shop_transactions LIMIT 10
--SELECT * FROM fct_coffee_shop_stores_reviews LIMIT 10
--SELECT * FROM dim_coffee_shop_stores LIMIT 10
--SELECT * FROM dim_coffee_products  LIMIT 10

In [0]:
%sql
SELECT 
  store_name,
  product_detail,
  make_date(year(transaction_date), month(transaction_date), day(transaction_date)) AS month_day,
  SUM(transaction_id) AS total_transactions,
  ROUND(sum(transation_total),2) AS total_sales,
  SUM(transaction_qty) AS total_quantity_sold
FROM 
  fct_coffee_shop_transactions
LEFT JOIN dim_coffee_shop_stores 
  ON fct_coffee_shop_transactions.store_id = dim_coffee_shop_stores.store_id
LEFT JOIN dim_coffee_products 
  ON fct_coffee_shop_transactions.product_id = dim_coffee_products.product_id
WHERE
  YEAR(transaction_date) = 2023
GROUP BY 
  store_name,
  product_detail,
  month_day
ORDER BY 
  store_name,
  month_day



In [0]:
%sql
SELECT
  product_detail,
  SUM(transaction_qty) AS total_quantity_sold
FROM
  fct_coffee_shop_transactions AS transactions
LEFT JOIN dim_coffee_products AS products 
  ON transactions.product_id = products.product_id
GROUP BY
  product_detail
ORDER BY
 total_quantity_sold DESC
 limit 10;

In [0]:
%sql


In [0]:
%sql
SELECT
  store_name,
  transaction_date,
  COUNT(transaction_id) AS total_transactions,
  ROUND(SUM(transation_total), 2) AS total_sales
FROM
  fct_coffee_shop_transactions AS transactions
LEFT JOIN dim_coffee_shop_stores AS stores
  ON transactions.store_id = stores.store_id
GROUP BY
  store_name,
  transaction_date
ORDER BY
  store_name,
  transaction_date

In [0]:
%sql
SELECT
  store_name,
  HOUR(transaction_time) AS hour_of_day,
  SUM(transaction_qty) AS total_transactions_per_hour,
  ROUND(SUM(transation_total),2) AS total_store_sales_per_hour
FROM
  fct_coffee_shop_transactions AS transactions
LEFT JOIN dim_coffee_shop_stores AS stores
  ON transactions.store_id = stores.store_id 
GROUP BY
  HOUR(transaction_time), store_name
ORDER BY
 store_name, hour_of_day ASC


In [0]:
%sql
SELECT
  store_name,
  HOUR(transaction_time) AS hour_of_day,
  COUNT(transaction_id) AS total_transactions,
  ROUND(SUM(transation_total), 2) AS total_sales
FROM
  fct_coffee_shop_transactions AS transactions
LEFT JOIN dim_coffee_shop_stores AS stores
  ON transactions.store_id = stores.store_id
GROUP BY
  store_name,
  hour_of_day
ORDER BY
  store_name,
  hour_of_day ASC

In [0]:
%sql
SELECT 
  store_name,
  COUNT(review_id) AS total_reviews
FROM 
  fct_coffee_shop_stores_reviews AS reviews
LEFT JOIN dim_coffee_shop_stores AS stores
  ON reviews.store_id = stores.store_id
GROUP BY 
  store_name
ORDER BY 
  total_reviews DESC



## Batch Inference: "SQL AI functions"
Let's leverage Batch Inference to give a more personalized description and to review the current data by analyzing the sentiment of the reviews

<div style="display: flex; align-items: center;">
  <img src="Images/Coffee and AI.png" alt="generated-image.png" style="margin-left: 10px;" width="300"/>
</div>


In [0]:
%sql
--CREATE OR REPLACE TABLE users.fernando_vasquez.dim_coffee_products_extended AS (
WITH distict_products AS (
  SELECT DISTINCT 
  product_id,
  product_category,
  product_detail,
  unit_price
FROM dim_coffee_products 
)
SELECT product_id,product_category,product_detail,
      ai_query('databricks-meta-llama-3-3-70b-instruct',
    CONCAT(
      "Create a description of the product based on the product category",product_category,"and detail",product_detail,"and unit price in dollars",unit_price,"The output should be just the description and vary for each product")) AS PRODUCT_DESCRIPTION
 FROM distict_products 
 ORDER BY product_id DESC
 LIMIT 35
--)

In [0]:
%sql
CREATE OR REPLACE TABLE fct_coffee_shop_stores_reviews AS (
  SELECT 
    review_id,
    store_id,
    email,
    review,
    score, 
    ai_analyze_sentiment(review) AS sentiment
  FROM 
    fct_coffee_shop_stores_reviews
) 

In [0]:
%sql
SELECT 
  sentiment,
  count(review_id) AS total_reviews
FROM 
  fct_coffee_reviews_sentiment
GROUP BY 
  sentiment

In [0]:
%sql
WITH reviews_combined  AS (
SELECT 
  fct_reviews.review_id,
  review,
  email,
  sentiment,
  store_name
FROM 
  fct_coffee_shop_stores_reviews AS fct_reviews 
LEFT JOIN dim_coffee_shop_stores AS dim_stores ON fct_reviews.store_id = dim_stores.store_id 
)

SELECT 
  review_id, 
  email,
      ai_gen(concat("Use this review:",review,"to create a email advertising a 10% discount at the store:",store_name,"Use the email:", email, "as to infer the user name",". The output should be just the email and vary for each review")) AS advertising_emails
FROM 
  reviews_combined
WHERE 
  sentiment = "mixed" limit 10


## Dashboard: "Tables to visualizations"
Now that we understand the data, and move the insights to production!

<div style="display: flex; align-items: right;">
  <img src="Images/Coffee Sales.png" alt="Coffee Sales" style="margin-left: 10px;" width="300"/>
</div>



## Genie: "powerful text to insights assistant"
Convert your business questions and last-mile analytics to SQL easily with Genie!


<div style="display: flex; align-items: right;">
 <img src="Images/Genie.png" alt="Genie" style="margin-left: 10px;" width="400"/>
</div>

### Here we will consider best practices when setting up the genie space:

* **Focused Datasets:** Genie should be product/project focus.
* **Leverage UC metadata:** Give proper table and column descriptions.
* **Give Genie SQL examples:** Add a natural language question that reflects what a business user would ask.
* **Add Joins:** Explain to gneie the relationship of your data.
* **Add instructions:** Help Genie understand the business context on meanings, formatting, and business rules.


In this workshop, we covered the first 2 Best practices at the beginning when we covered the UC tables.

### Lets create a Genie Space to work as a  Coffee Information Asistant 

#### **Description:** 
This assistant will use data about the transactions, reviews, and sales of coffee stores. Additionally, it includes information about the different products available in the chain.

#### **Guide the users:** 

For example, SQL, let's review a couple of questions

#### **Instructions:** 

* Adress to the user in the most formal way 
* Any product with total sales bellow 800 is at risk of beeing removed from the Menu
* Stores should consider negative very seriusly 
* Hours with less than 10000 total sells could be removed 
* Customer satisfaction its very important for corportate
* Better customer experience is more important than total sells
* All the data related to money (Like sales or price) should be assumed as in dollars 
* The best product is the one with the highest sales

#### **Sample questions:** 
* What is the best time selling time for the chain?
* How much reviews have a negative and whats their average score?
* What products should we remove?



#### **Joins:** 
Define the 3 joins shown on this Entity Relatioship Diagram 


<div style="display: flex; align-items: right;">
 <img src="Images/Coffee EDR.png" alt="Genie" style="margin-left: 10px;" width="600"/>
</div>


#### **Example, SQL:** 


In [0]:
%sql
--What is the top 3 best selling product for the store Astoria Brews store?
--This is relevant because it helps us understand what products are popular with customers and for marketing strategies.
SELECT
  store_name,
  product_detail,
  SUM(transation_total) AS total_quantity_sold
FROM
  fct_coffee_shop_transactions AS transactions
LEFT JOIN dim_coffee_products AS products 
  ON transactions.product_id = products.product_id
LEFT JOIN dim_coffee_shop_stores AS stores
  ON transactions.store_id = stores.store_id
WHERE
  store_name = 'Astoria Brews'
GROUP BY
  store_name,
  product_detail
ORDER BY
  total_quantity_sold DESC
LIMIT 3


In [0]:
%sql
--What's the most crowded time for Lower Grounds Coffee Co?  
--This is relevant because it helps us understand when the store is most busy and can help us plan for staffing
SELECT
  store_name,
  HOUR(transaction_time) AS hour_of_day,
  ROUND(SUM(transation_total),2) AS total_transactions
FROM
  fct_coffee_shop_transactions AS transactions
LEFT JOIN dim_coffee_shop_stores AS stores
  ON transactions.store_id = stores.store_id
WHERE
  store_name = 'Lower Grounds Coffee Co'
GROUP BY  store_name, hour_of_day

ORDER BY
  total_transactions DESC
LIMIT 1